In [ ]:
import pandas as pd

data = pd.read_csv('drive/My Drive/Colab Notebooks/titanicdata/train.csv')
print(data)

print(data.isnull().sum())

average = data['Age'].mean()
print(average)

mode = data['Embarked'].mode()

data['Age'].fillna(value=30, inplace=True)
data['Embarked'].fillna(value='S', inplace=True)

print(data.isnull().sum())

In [9]:
import tensorflow as tf

key = data.pop('Survived')

ds = tf.data.Dataset.from_tensor_slices( (dict(data), key) )
print(ds)


<_TensorSliceDataset element_spec=({'PassengerId': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Pclass': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Name': TensorSpec(shape=(), dtype=tf.string, name=None), 'Sex': TensorSpec(shape=(), dtype=tf.string, name=None), 'Age': TensorSpec(shape=(), dtype=tf.float64, name=None), 'SibSp': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Parch': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Ticket': TensorSpec(shape=(), dtype=tf.string, name=None), 'Fare': TensorSpec(shape=(), dtype=tf.float64, name=None), 'Embarked': TensorSpec(shape=(), dtype=tf.string, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [12]:
feature_columns=[ ]

def norm(x):
  min = data['Fare'].min()
  max = data['Fare'].max()
  return (x - min) / (max - min) #compress fare into 0 to 1

# put as integers itself
feature_columns.append( tf.feature_column.numeric_column('Fare', normalizer_fn= norm ) )
feature_columns.append( tf.feature_column.numeric_column('Parch') )
feature_columns.append( tf.feature_column.numeric_column('SibSp') )

# put as groups=bucketized, 10s, 20s, 30s, ...
Age = tf.feature_column.numeric_column('Age')
Age_bucket = tf.feature_column.bucketized_column(Age, boundaries=[10,20,30,40,50,60] )
feature_columns.append( Age_bucket )

# put as categorized = one hot encoding, not many kinds
vocab = data['Sex'].unique()
cat = tf.feature_column.categorical_column_with_vocabulary_list('Sex', vocab)
one_hot = tf.feature_column.indicator_column(cat)
feature_columns.append( one_hot )

vocab = data['Embarked'].unique()
cat = tf.feature_column.categorical_column_with_vocabulary_list('Embarked', vocab)
one_hot = tf.feature_column.indicator_column(cat)
feature_columns.append( one_hot )

vocab = data['Pclass'].unique()
cat = tf.feature_column.categorical_column_with_vocabulary_list('Pclass', vocab)
one_hot = tf.feature_column.indicator_column(cat)
feature_columns.append( one_hot )

# put as embedding = too many kinds
vocab = data['Ticket'].unique()
cat = tf.feature_column.categorical_column_with_vocabulary_list('Ticket', vocab)
embedding = tf.feature_column.embedding_column(cat, dimension=9)
feature_columns.append( embedding )


Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.k

In [16]:
ds_batch = ds.batch(32)

feature_layer = tf.keras.layers.DenseFeatures( tf.feature_column.numeric_column('Fare') )
feature_layer(next(iter(ds_batch))[0])




<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[  7.25  ],
       [ 71.2833],
       [  7.925 ],
       [ 53.1   ],
       [  8.05  ],
       [  8.4583],
       [ 51.8625],
       [ 21.075 ],
       [ 11.1333],
       [ 30.0708],
       [ 16.7   ],
       [ 26.55  ],
       [  8.05  ],
       [ 31.275 ],
       [  7.8542],
       [ 16.    ],
       [ 29.125 ],
       [ 13.    ],
       [ 18.    ],
       [  7.225 ],
       [ 26.    ],
       [ 13.    ],
       [  8.0292],
       [ 35.5   ],
       [ 21.075 ],
       [ 31.3875],
       [  7.225 ],
       [263.    ],
       [  7.8792],
       [  7.8958],
       [ 27.7208],
       [146.5208]], dtype=float32)>

In [15]:
model = tf.keras.Sequential([
    
    tf.keras.layers.DenseFeatures(feature_columns),

    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid'), # 1 = survived, 0 = dead
    
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

ds_batch = ds.batch(32)

model.fit(ds_batch, shuffle=True, epochs=20)

Epoch 1/20


28/28 [==============================] - 4s 4ms/step - loss: 1.0935 - acc: 0.5455
Epoch 2/20
28/28 [==============================] - 0s 4ms/step - loss: 0.7837 - acc: 0.6813
Epoch 3/20
28/28 [==============================] - 0s 4ms/step - loss: 0.7249 - acc: 0.6970
Epoch 4/20
28/28 [==============================] - 0s 4ms/step - loss: 0.6060 - acc: 0.7441
Epoch 5/20
28/28 [==============================] - 0s 4ms/step - loss: 0.5907 - acc: 0.7856
Epoch 6/20
28/28 [==============================] - 0s 3ms/step - loss: 0.5041 - acc: 0.7924
Epoch 7/20
28/28 [==============================] - 0s 4ms/step - loss: 0.5748 - acc: 0.7823
Epoch 8/20
28/28 [==============================] - 0s 4ms/step - loss: 0.4493 - acc: 0.8204
Epoch 9/20
28/28 [==============================] - 0s 4ms/step - loss: 0.4737 - acc: 0.8171
Epoch 10/20
28/28 [==============================] - 0s 4ms/step - loss: 0.4613 - acc: 0.8047
Epoch 11/20
28/28 [==============================] - 0s 4ms/step - loss: 0.3886 